In [13]:
import os
import sys
sys.path.append('../src')

from openai import OpenAI
from dotenv import load_dotenv
from agents import BaseAgent, Agent, MultiAgent
from pydantic import BaseModel

In [14]:
load_dotenv()

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)


In [15]:
# Top level agent
general_agent = Agent(base_agent = BaseAgent(client),
           name = "general agent",
           instructions= "Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
           handoff_description = "Call this agent if you don't know how to answer the user's question or do not has access to the necessary information.")

In [16]:
class AgentDesc(BaseModel):
    name: str
    instructions: str
    handoff_description: str
    
def create_new_agent(input: str)->Agent:
    """ 
    If there is no other agent or tool can handle the input, call this function to create a new agent.
    """

    model = "gpt-4o-mini"

    completion = client.beta.chat.completions.parse(
    model=model,
    messages=[
        {"role": "system", "content": """ 
         Create a new agent. The agent should be able to handle user request. return the agent description.

        ## name example
        
         science agent (only english characters and spaces are allowed)
         
        ## instructions example
         
        You are a Science Agent As a science agent, you have a strong understanding of physics, chemistry, and biology. You can answer a variety of questions in these fields. Your aim is to make science interesting and easy to understand for everyone.
         
        ## handoff_description example
        
        Call this agent if you have a question related to science, physics, chemistry, or biology. This agent can provide detailed explanations, clarify complex concepts, and offer insights into the natural world.
         
         """},
        {"role": "user", "content": input},
    ],
    response_format=AgentDesc,
)

    event = completion.choices[0].message.parsed

    new_agent = Agent(base_agent = BaseAgent(client),name=event.name, instructions=event.instructions, handoff_description=event.handoff_description)

    general_agent.add_handoff(new_agent)
    new_agent.add_handoff(general_agent)

    return "#CreateNewAgent#"
    


In [17]:
general_agent.add_tools([create_new_agent])

In [18]:
multi_agent = MultiAgent(start_agent=general_agent)

In [19]:
multi_agent.chat([{ "role": "user", "content": "how to ride a bike?"}],show_details=True)

ERROR:root:Unable to generate ChatCompletion response
ERROR:root:Exception: 'Agent' object has no attribute 'add_handoff'


In [9]:
multi_agent.chat([{ "role": "user", "content": "recommend a bike for me?"}],show_details=True)

[{'role': 'assistant',
  'content': 'To recommend the best bike for you, I need some additional information:\n\n1. **Purpose**: What will you primarily use the bike for (e.g., commuting, leisure riding, mountain biking, racing)?\n2. **Terrain**: What kind of terrain will you be riding on (e.g., paved roads, gravel paths, trails)?\n3. **Experience Level**: Are you a beginner, intermediate, or advanced rider?\n4. **Budget**: What is your budget range for the bike?\n5. **Preferred Style**: Do you have a preference for bike type (e.g., road bike, mountain bike, hybrid, cruiser)?\n6. **Size**: Do you know what size bike you need (usually based on your height)?\n\nOnce you provide this information, I can give you a more tailored recommendation!',
  'sender': 'Bike_Riding_Agent'}]

In [11]:
multi_agent.chat([{ "role": "user", "content": "can you explain the theory of relativity?"}],show_details=True)

[{'role': 'assistant',
  'tool_calls': [{'id': 'call_KDFjNF4S6lJidLB4aa61vX44',
    'function': {'arguments': '{"message": "can you explain the theory of relativity?"}',
     'name': 'general_agent'},
    'type': 'function'}],
  'type': 'handoff',
  'sender': 'Bike_Riding_Agent'},
 {'role': 'tool',
  'handoff': 'Bike_Riding_Agent -> general_agent',
  'content': 'Handing off to general_agent',
  'tool_call_id': 'call_KDFjNF4S6lJidLB4aa61vX44',
  'message': 'can you explain the theory of relativity?',
  'type': 'handoff',
  'sender': 'general_agent'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_tJhhFQz7bvRP9vKdQwvcEVVm',
    'function': {'arguments': '{"input":"can you explain the theory of relativity?"}',
     'name': 'create_new_agent'},
    'type': 'function'}],
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'tool',
  'content': '#CreateNewAgent#',
  'tool_call_id': 'call_tJhhFQz7bvRP9vKdQwvcEVVm',
  'type': 'tool',
  'sender': 'general_agent'},
 {'role': 'assistan

In [12]:
general_agent.handoff_agent_schemas

[{'type': 'function',
  'function': {'name': 'Relativity_Agent',
   'description': 'You are a Relativity Agent. As a relativity agent, you specialize in the theory of relativity, encompassing both special and general relativity. You are able to explain complex concepts such as time dilation, the speed of light, the warping of space-time, and the implications of these theories on our understanding of the universe.',
   'parameters': {'type': 'object',
    'properties': {'message': {'type': 'string',
      'description': 'The message to send to the agent.'}},
    'required': ['message']}}}]